# Precinct Name Cleanup

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
%load_ext autoreload
%autoreload 2

In [3]:
from data.voterdb import VoterDb

In [4]:
root_dir = '~/Documents/data'
db = VoterDb(root_dir)

Get VTD precinct ids and names.
Get election result county and precinct name. Drop duplicates.
Get county details and merge county code with election results.
Rename county to county name in election result.
Merge election results with county details to add county code.
Merge election result with vtd to get precinct ids
Identify missing precinct ids


In [5]:
vtd_p = pd.read_sql_query('select county_code, precinct_id, precinct_name from vtd_map', db.con)
er_p = pd.read_sql_query('select county, precinct_name from election_results', db.con).drop_duplicates().reset_index()
er_p = er_p.rename(columns={'county': 'county_name'})
cd = db.county_details
er_p = er_p.merge(cd, on='county_name', how='left')
er_p = er_p.merge(vtd_p, on=['county_code', 'precinct_name'], how='left')[['county_code', 'county_name', 'precinct_id', 'precinct_name']]
er_p = er_p.sort_values(['county_name', 'precinct_name'])
er_p_missing = er_p[er_p.precinct_id.isna()]

In [25]:
from thefuzz import process
results = []
for c in er_p_missing.county_code.unique():
    choices = vtd_p[vtd_p.county_code == c].precinct_name
    for x in er_p_missing[er_p_missing.county_code == c].precinct_name:
        choice, score, _ = process.extractOne(x, choices)
        results.append((c, x, choice, score))



In [22]:
len(list(filter(lambda x: x[2] >= 85, results)))

341

In [27]:
list(filter(lambda x: x[0]=='067', sorted(results, key=lambda x: x[2], reverse=True)))

[('067', 'PINCKNEYVILLE A', 'PINCKNEYVILLE A1', 97)]

In [30]:
db.county_details.to_csv('../resources/county_details.csv', index=False)

## Baldwin County

MERIWEATHER should be MERIWETHER
SCOTTBORO should be SCOTTSBORO
WEST THOMAS is missing from VTD

In [53]:
vtd_p[vtd_p.county_code=='005']

,county_code,precinct_id,precinct_name
981,005,322,COOPERS
984,005,1714,SCOTTSBORO
985,005,CTY6,NORTH MILLEDGEVILLE
986,005,MONT,EAST BALDWIN
991,005,CTY2,CITY HALL ANNEX
992,005,321E,EAST HARDWICK
993,005,CTY3,BODDIE
994,005,319,WEST BALDWIN
995,005,MERI,MERIWEATHER
996,005,321W,WEST HARDWICK


## Bryan County

JF GREGORY PARK = J.F.GREGORY PARK
NEW BEGINNINGS is missing
RH RECREATION COMPX = RH REC COMPLEX

In [54]:
vtd_p[vtd_p.county_code=='015']

,county_code,precinct_id,precinct_name
786,015,9,PUBLIC SAFETY COMPX
787,015,5,RH RECREATION COMPX
788,015,6,J.F.GREGORY PARK
1216,015,3,BLACK CREEK
1259,015,8,HWY 144 EAST
1264,015,7,KELLER
1265,015,10,DANIELSIDING
1266,015,4,WAYS STATION
1292,015,1,PEMBROKE
1294,015,2,ELLABELL


In [16]:
df[df.county_code == '060'].sort_values(['precinct_id'])

,county_code,precinct_id,precinct_name
2196,060,01A,01A
2410,060,01B,01B
2282,060,01C,01C
2472,060,01D,01D
2206,060,01E,01E
...,...,...,...
2520,060,UC02B,UC02B
2265,060,UC02C,UC02C
2276,060,UC031,UC031
2347,060,UC032,UC032


In [18]:
df_e = df_e.rename(columns={'county': 'county_name'})

In [19]:
cd = db.county_details
df_e = df_e.merge(cd, on='county_name', how='left')

In [20]:
df_e.county_code.isna().sum()

0

In [29]:
df1 = df_e[['county_name', 'county_code', 'precinct_name']].drop_duplicates().merge(df_p, on=['county_code', 'precinct_name'], how='left')

In [30]:
df1.precinct_id.isna().sum()

472

In [31]:
df_e.shape

(324238, 4)

In [32]:
x = df1[df1.precinct_id.isna()]
x

,county_name,county_code,precinct_name,precinct_id
10,TROUP,141,MCCLENDON,NaN
20,CHARLTON,024,WINKOUR,NaN
31,FLOYD,057,FOSTERS MILL,NaN
65,DOUGLAS,048,EPHESUS BAPTIST CHURCH,NaN
69,DOUGLAS,048,GOLDEN METHODIST CHURCH,NaN
...,...,...,...,...
2655,LAURENS,087,FBC-FLC,NaN
2656,LAURENS,087,FIRE DEPT STATION 5,NaN
2658,LAURENS,087,LCFS #10,NaN
2661,LAURENS,087,RURAL FIRE STATION 7,NaN


In [33]:
df_p[df_p.county_code=='058']

,county_code,precinct_id,precinct_name
64,058,35,35 JOHNS CREEK
1381,058,04,04 CHESTATEE
1472,058,36,36 NICHOLS
1683,058,19,19 OLD ATLANTA
1780,058,02,02 BRANDYWINE
1798,058,34,34 FOWLER
1801,058,01,01 BIG CREEK
1802,058,21,21 SOUTH FORSYTH
1803,058,29,29 POLO
1804,058,10,10 MIDWAY


In [40]:
x = x.sort_values(['county_name'])
x

,county_name,county_code,precinct_name,precinct_id
142,BALDWIN,005,WEST THOMAS,NaN
138,BALDWIN,005,SCOTTBORO,NaN
135,BALDWIN,005,MERIWETHER,NaN
1604,BRYAN,015,NEW BEGINNINGS,NaN
1606,BRYAN,015,RH REC COMPLEX,NaN
...,...,...,...,...
563,WARE,148,408 - MILLWOOD BAPTIST CHURCH,NaN
557,WARE,148,304 - WARE BOE LEARNING CENTER,NaN
124,WHITE,154,ROBERTSTOWN-HELEN,NaN
2184,WILKES,157,YOUNG FAMER'S BLDG,NaN
